In [ ]:
# !conda install tabula
# !conda install pdfplumber

# LOAD PACKAGES

In [167]:
from typing import List, Dict, Optional, Union
import pandas as pd
import numpy as np
import importlib
import re
from tqdm import tqdm
from urllib.parse import urljoin
import warnings
warnings.filterwarnings("ignore")

# Python Package to extract and automate webscraping
import requests
from bs4 import BeautifulSoup
import time
import os

# Python Package to extract pdf tables
import tabula
import pdfplumber

## WEBSCRAPE

In [30]:
# Install required packages if not already installed
# !pip install beautifulsoup4 requests pandas tqdm

# URL to scrape
url = "https://pantawid.dswd.gov.ph/programimplementationreport/"

# Create a directory to store downloaded files
download_dir = "pantawid_reports"
os.makedirs(download_dir, exist_ok=True)

# Send a request to the website with proper headers to mimic a browser
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def download_file(url, filename):
    """Download a file from a URL to a specified filename"""
    try:
        response = requests.get(url, headers=headers, stream=True)
        response.raise_for_status()  # Raise an exception for HTTP errors
        
        # Get file size for progress bar
        file_size = int(response.headers.get('Content-Length', 0))
        
        # Show download progress
        progress_bar = tqdm(total=file_size, unit='B', unit_scale=True, desc=filename)
        
        with open(filename, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    file.write(chunk)
                    progress_bar.update(len(chunk))
        
        progress_bar.close()
        return True
    except Exception as e:
        print(f"Error downloading {url}: {e}")
        return False

try:
    # Make the request to the main page
    response = requests.get(url, headers=headers)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find all anchor tags (links)
        links = soup.find_all('a')
        
        # Filter links that match the quarterly report pattern
        quarterly_reports = []
        
        for link in links:
            href = link.get('href', '')
            text = link.text.strip()
            
            # Check if the link points to a PDF file and contains quarterly report keywords
            if href.endswith('.pdf') and re.search(r'quarter|quarterly|q[1-4]|report', href.lower() + ' ' + text.lower(), re.IGNORECASE):
                # Make sure we have the full URL
                full_url = urljoin(url, href)
                
                # Extract year from the URL or text if possible
                year_match = re.search(r'20\d{2}', href + ' ' + text)
                year = year_match.group(0) if year_match else "Unknown"
                
                # Extract quarter information
                quarter = ""
                if '1st' in text.lower() or 'first' in text.lower() or 'q1' in text.lower():
                    quarter = "1st Quarter"
                elif '2nd' in text.lower() or 'second' in text.lower() or 'q2' in text.lower():
                    quarter = "2nd Quarter"
                elif '3rd' in text.lower() or 'third' in text.lower() or 'q3' in text.lower():
                    quarter = "3rd Quarter"
                elif '4th' in text.lower() or 'fourth' in text.lower() or 'q4' in text.lower():
                    quarter = "4th Quarter"
                else:
                    quarter = "Quarterly Report"
                
                quarterly_reports.append({
                    'Quarter': quarter,
                    'Year': year,
                    'Link Text': text,
                    'URL': full_url
                })
        
        # If no quarterly reports found, look for any PDF files
        if not quarterly_reports:
            print("No specific quarterly reports found. Looking for any PDF files...")
            for link in links:
                href = link.get('href', '')
                text = link.text.strip()
                
                if href.endswith('.pdf'):
                    full_url = urljoin(url, href)
                    quarterly_reports.append({
                        'Quarter': 'Unknown',
                        'Year': 'Unknown',
                        'Link Text': text,
                        'URL': full_url
                    })
        
        # Create a DataFrame from the collected reports
        if quarterly_reports:
            df = pd.DataFrame(quarterly_reports)
            
            print(f"Found {len(df)} report files to download.")
            
            # Download each file
            for index, row in df.iterrows():
                # Create a filename based on the report information
                if row['Year'] != 'Unknown' and row['Quarter'] != 'Unknown':
                    filename = f"{row['Year']}_{row['Quarter'].replace(' ', '_')}.pdf"
                else:
                    # Extract filename from URL if year/quarter unknown
                    url_filename = os.path.basename(row['URL'])
                    filename = url_filename if url_filename else f"report_{index}.pdf"
                
                # Full path for saving
                filepath = os.path.join(download_dir, filename)
                
                print(f"\nDownloading: {row['Link Text']} ({row['URL']})")
                success = download_file(row['URL'], filepath)
                
                if success:
                    print(f"Successfully downloaded to {filepath}")
                else:
                    print(f"Failed to download {row['URL']}")
            
            print(f"\nAll downloads completed. Files saved to {os.path.abspath(download_dir)}")
        else:
            print("No PDF reports found on the page.")
    else:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")

except Exception as e:
    print(f"An error occurred: {e}")

Found 50 report files to download.

Downloading: 1st Quarter Report (https://pantawid.dswd.gov.ph/wp-content/uploads/2025/06/First-Quarter-PIR_2025.pdf)


pantawid_reports/2025_1st_Quarter.pdf: 5.87MB [00:00, 15.9MB/s]


Successfully downloaded to pantawid_reports/2025_1st_Quarter.pdf

Downloading: 2nd Quarter Report (https://pantawid.dswd.gov.ph/wp-content/uploads/2025/08/2nd-Quarter-Program-Implementation-Report.pdf)


pantawid_reports/2025_2nd_Quarter.pdf: 4.40MB [00:00, 12.2MB/s]


Successfully downloaded to pantawid_reports/2025_2nd_Quarter.pdf

Downloading: 1st Quarter Report (https://pantawid.dswd.gov.ph/wp-content/uploads/2024/08/2024-1st-Quarter-Report.pdf)


...

Successfully downloaded to pantawid_reports/2018_2nd_Quarter.pdf

Downloading: 3rd Quarter Report (https://pantawid.dswd.gov.ph/wp-content/uploads/2018/07/2012-3rd-Quarter2012.pdf)


pantawid_reports/2018_3rd_Quarter.pdf: 842kB [00:00, 4.86MB/s]


Successfully downloaded to pantawid_reports/2018_3rd_Quarter.pdf

Downloading: 4th Quarter Report (https://pantawid.dswd.gov.ph/wp-content/uploads/2018/07/2012-4th-Quarter2012.pdf)


pantawid_reports/2018_4th_Quarter.pdf: 830kB [00:00, 4.83MB/s]

Successfully downloaded to pantawid_reports/2018_4th_Quarter.pdf

All downloads completed. Files saved to /Users/christianromeoplan/Documents/Personal Projects/pantawid_reports


In [33]:
import os

# Path to the directory
pantawid_reports_dir = "pantawid_reports"

# Check if the directory exists
if os.path.exists(pantawid_reports_dir) and os.path.isdir(pantawid_reports_dir):
    # Get list of all files in the directory
    files = os.listdir(pantawid_reports_dir)
    
    # Print the number of files found
    print(f"Found {len(files)} files in {pantawid_reports_dir} directory:")
    
    # Loop through and print each filename
    for i, filename in enumerate(files, 1):
        # Get full path to check if it's a file (not a subdirectory)
        full_path = os.path.join(pantawid_reports_dir, filename)
        
        if os.path.isfile(full_path):
            # Get file size in KB
            file_size = os.path.getsize(full_path) / 1024
            
            # Print file information
            print(f"{i}. {filename} ({file_size:.2f} KB)")
            
            # If it's a PDF, you could extract more information
            if filename.lower().endswith('.pdf'):
                # Extract year and quarter from filename if available
                parts = filename.split('_')
                if len(parts) >= 2 and parts[0].isdigit():
                    year = parts[0]
                    quarter = parts[1].replace('_', ' ')
                    print(f"   Year: {year}, Quarter: {quarter}")
    
    # Alternative simple loop as requested
    print("\nSimple list of filenames:")
    for a in list(os.listdir(pantawid_reports_dir)):
        print(a)
else:
    print(f"Directory '{pantawid_reports_dir}' does not exist. Please run the download script first.")
    
    # Suggest creating the directory
    create_dir = input("Would you like to create the directory now? (y/n): ")
    if create_dir.lower() == 'y':
        os.makedirs(pantawid_reports_dir, exist_ok=True)
        print(f"Directory '{pantawid_reports_dir}' created. It's empty.")

Found 25 files in pantawid_reports directory:
1. 2022_1st_Quarter.pdf (7049.28 KB)
   Year: 2022, Quarter: 1st
2. 2021_3rd_Quarter.pdf (7755.16 KB)
   Year: 2021, Quarter: 3rd
3. 2025_1st_Quarter.pdf (5731.13 KB)
   Year: 2025, Quarter: 1st
4. 2018_1st_Quarter.pdf (773.39 KB)
   Year: 2018, Quarter: 1st
5. 2021_4th_Quarter.pdf (7220.94 KB)
   Year: 2021, Quarter: 4th
6. 2024_1st_Quarter.pdf (7047.08 KB)
   Year: 2024, Quarter: 1st
7. 2020_2nd_Quarter.pdf (667.99 KB)
   Year: 2020, Quarter: 2nd
8. 2020_4th_Quarter.pdf (580.48 KB)
   Year: 2020, Quarter: 4th
9. 2019_1st_Quarter.pdf (658.80 KB)
   Year: 2019, Quarter: 1st
10. 2020_3rd_Quarter.pdf (656.43 KB)
   Year: 2020, Quarter: 3rd
11. 2022_4th_Quarter.pdf (7713.70 KB)
   Year: 2022, Quarter: 4th
12. 2022_2nd_Quarter.pdf (7407.34 KB)
   Year: 2022, Quarter: 2nd
13. 2018_3rd_Quarter.pdf (821.94 KB)
   Year: 2018, Quarter: 3rd
14. 2022_3rd_Quarter.pdf (2643.83 KB)
   Year: 2022, Quarter: 3rd
15. 2021_1st_Quarter.pdf (7301.03 KB)
   Year

______
## LOAD FILE & SPECIFIC PAGES

In [228]:
PDF = "pantawid_reports/2025_4th_Quarter.pdf"
page_num = 22

_____________
## DATA EXTRACTION V1: TABULA EXTRACT TABLE

In [237]:
def extract_table_from_pdf_tabula(pdf_path, page_num, table_index=0):
    """
    Extracts a table from a given PDF and page using tabula.
    Returns the specified table as a DataFrame.
    table_index: If you have multiple tables in a pdf page, change this based on the order of tables. hehe
    """
    dfs = tabula.read_pdf(
        pdf_path,
        pages=page_num,
        multiple_tables=True,
        stream=False,   # set True if no visible grid lines
        guess=True
    )
    return dfs[table_index] if dfs else None


# Run Function
tabula_df = extract_table_from_pdf_tabula(PDF, page_num, table_index=1)
tabula_df.head(10)

Got stderr: 2025-10-29 12:45:10.967 java[80719:2019079] [JRSAppKitAWT markAppIsDaemon] failed. SetApplicationIsDaemon returned -50



,Unnamed: 0,Unnamed: 1,Household with Single,Unnamed: 2,Total,Percentage
0,NaN,Region,Multiple Disability Disability Disability,NaN,Member,%
1,NCR,NaN,"1,874 1,965",39,"2,004",3.97%
2,CAR,NaN,"1,138 1,150",54,"1,204",2.39%
3,I,NaN,"3,667 3,954",148,"4,102",8.13%
4,II,NaN,"2,175 2,233",80,"2,313",4.58%
5,III,NaN,"3,049 3,232",80,"3,312",6.56%
6,IV-A,NaN,"3,905 4,064",148,"4,212",8.35%
7,MIMAROPA,NaN,"2,997 3,010",174,"3,184",6.31%
8,V,NaN,"5,184 5,359",99,"5,458",10.82%
9,VI,NaN,"4,532 4,665",167,"4,832",9.58%


#### NOTES: Change table_index if you want to extract other tables on the same page

_____________
## DATA EXTRACTION V2: PDFPLUMBER EXTRACT TABLE

In [239]:
def extract_table_from_pdf_pdfplumber(pdf_path, page_num, table_index=0):
    """
    Extracts tables from a given PDF page using pdfplumber.
    Returns the specified table as a DataFrame.
    """
    with pdfplumber.open(pdf_path) as pdf:
        page = pdf.pages[page_num - 1]  # pdfplumber uses 0-based indexing
        tables = page.extract_tables()

    dfs = [pd.DataFrame(t) for t in tables if t and any(any(cell for cell in row) for row in t)]

    if not dfs:
        print(f"No tables found on page {page_num}.")
        return None

    # Combine all tables from that page
    df_all = pd.concat(dfs, ignore_index=True)

    return dfs[table_index] if len(dfs) > table_index else df_all


# Run Funciton
pdfplumber_df = extract_table_from_pdf_pdfplumber(PDF, page_num, table_index=1)
pdfplumber_df.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,Region,None,None,,Household with,,,Single,,Multiple Disability,None,None,,Total,,,Percentage,
1,None,None,None,None,Disability,None,None,Disability,None,None,None,None,None,Member,None,None,%,None
2,NCR,None,None,"1,874",None,None,"1,965",None,None,39,None,None,"2,004",None,None,3.97%,None,None
3,CAR,None,None,"1,138",None,None,"1,150",None,None,54,None,None,"1,204",None,None,2.39%,None,None
4,I,None,None,"3,667",None,None,"3,954",None,None,148,None,None,"4,102",None,None,8.13%,None,None
5,II,None,None,"2,175",None,None,"2,233",None,None,80,None,None,"2,313",None,None,4.58%,None,None
6,III,None,None,"3,049",None,None,"3,232",None,None,80,None,None,"3,312",None,None,6.56%,None,None
7,IV-A,None,None,"3,905",None,None,"4,064",None,None,148,None,None,"4,212",None,None,8.35%,None,None
8,MIMAROPA,None,None,"2,997",None,None,"3,010",None,None,174,None,None,"3,184",None,None,6.31%,None,None
9,V,None,None,"5,184",None,None,"5,359",None,None,99,None,None,"5,458",None,None,10.82%,None,None


____________
## DATA PREPROCESSING STEP 1: CONSOLIDATE COLUMNS

In [199]:
def combine_every_3_columns(df):
    """
    Combines every 3 consecutive columns into one.
    If any of the three columns has a non-empty value, that value is kept.
    """
    combined_cols = []
    num_cols = df.shape[1]

    for i in range(0, num_cols, 3):
        # select a block of 3 columns
        block = df.iloc[:, i:i+3]
        combined = block.apply(lambda row: next((x for x in row if pd.notna(x) and str(x).strip() != ""), None), axis=1)
        combined_cols.append(combined)

    # stack all combined columns horizontally
    combined_df = pd.concat(combined_cols, axis=1)

    # assign column names
    combined_df.columns = [f"group_{i+1}" for i in range(combined_df.shape[1])]

    return combined_df


# Run Finction
cleaning_step_1_df = combine_every_3_columns(pdfplumber_df)
cleaning_step_1_df.head(10)

,group_1,group_2,group_3,group_4,group_5,group_6,group_7,group_8
0,Island / Region,Age Group (Years Old),None,None,None,None,Total,Percentage\n(%)
1,None,9 to 17,18 to 25,26 to 35,36 to 45,46 and,None,None
2,None,None,None,None,None,above,None,None
3,NCR,2,38,149,100,16,305,1.36%
4,CAR,24,113,322,211,3,673,3.00%
5,I,37,183,487,290,8,"1,005",4.48%
6,II,15,53,304,137,6,515,2.30%
7,III,20,100,444,274,17,855,3.81%
8,IV-A,16,168,"1,039",625,48,"1,896",8.45%
9,MIMAROPA,13,57,359,204,4,637,2.84%


_________
## DATA PREPROCESSING STEP 2: EXTRACT & CLEAN COLUMN NAMES

________
### 2.1: Find the NCR row and the rows before it (usually headers/group labels)

In [200]:
def get_rows_before_ncr(df: pd.DataFrame):
    """Return (ncr_idx, before_ncr). Uses the first column; NCR is the anchor.
    Since NCR always appears first in the report, we use it as the anchor (basis) row.
    This function returns:
      1) ncr_idx  - the row index where 'NCR' appears
      2) before_ncr - all rows before 'NCR', usually headers or grouping info
    """
    ncr_idx = df[df.iloc[:, 0] == "NCR"].index[0]
    before_ncr = df.iloc[:ncr_idx]
    return ncr_idx, before_ncr


# Run Finction
ncr_idx, before_ncr = get_rows_before_ncr(cleaning_step_1_df)
print("Index of NCR in this table is: ", ncr_idx)
pd.DataFrame(before_ncr)

Index of NCR in this table is:  3


,group_1,group_2,group_3,group_4,group_5,group_6,group_7,group_8
0,Island / Region,Age Group (Years Old),None,None,None,None,Total,Percentage\n(%)
1,None,9 to 17,18 to 25,26 to 35,36 to 45,46 and,None,None
2,None,None,None,None,None,above,None,None


_________
### 2.2: Count non-empty cells per column AFTER the first row (to know if we should combine rows or keep first row)

In [201]:
def count_non_empty_after_first(before_ncr: pd.DataFrame) -> pd.Series:
    """Count non-empty (not None/'None'/NaN/blank) cells per column after the first row."""
    return before_ncr.iloc[1:].apply(
        lambda col: (col.astype(str)
                       .str.strip()
                       .replace(["None","none","NaN","nan","NULL","null",""], pd.NA)
                       .notna()
                       .sum())
    )

# Run function
counts = count_non_empty_after_first(before_ncr)
counts

group_1    0
group_2    1
group_3    1
group_4    1
group_5    1
group_6    2
group_7    0
group_8    0
dtype: int64

__________
### 2.3: Build the final header names using the rule:
* if no non-empty below the first row -> use row 0 only
* otherwise -> combine all rows after row 0

________
#### 2.3.1: Clean and join a column’s values (replace newlines, trim, drop None/'None'/NaN/blanks)

In [ ]:
def clean_and_join(series: pd.Series) -> str | None:
    """Combine non-empty cells into one string (newline->space, trimmed). Return None if nothing left."""
    cleaned = [
        str(x).replace("\n", " ").strip()
        for x in series
        if pd.notna(x) and str(x).strip().lower() != "none" and str(x).strip() != ""
    ]
    return " ".join(cleaned) if cleaned else None

________
#### 2.3.2: Build the final headers

In [202]:
def build_headers(before_ncr: pd.DataFrame, counts: pd.Series) -> list[str]:
    """Return a list of new column headers following the 'keep-first-or-combine-rest' rule."""
    new_headers = []
    for pos, (col, count) in enumerate(zip(before_ncr.columns, counts)):
        if count == 0:
            new_headers.append(before_ncr.iloc[0, pos])
        else:
            new_headers.append(clean_and_join(before_ncr[col].iloc[1:]))
    return new_headers


# Run the function
new_headers = build_headers(before_ncr, counts)
new_headers

['Island / Region',
 '9 to 17',
 '18 to 25',
 '26 to 35',
 '36 to 45',
 '46 and above',
 'Total',
 'Percentage\n(%)']

_________
## DATA PREPROCESSING STEP 3: APPLY CLEANED HEADERS TO ACTUAL DATA

In [206]:
def apply_headers(final_df: pd.DataFrame, ncr_idx: int, new_headers: list[str]) -> pd.DataFrame:
    """Slice data from NCR onward and assign the computed headers."""
    clean_df = final_df.iloc[ncr_idx:].copy()
    clean_df.columns = new_headers
    return clean_df



# Run function
cleaning_step_2_df = apply_headers(cleaning_step_1_df, ncr_idx, new_headers)
cleaning_step_2_df.tail(15)

,Island / Region,9 to 17,18 to 25,26 to 35,36 to 45,46 and above,Total,Percentage\n(%)
10,V,23,152,"1,137",862,41,"2,215",9.88%
11,Luzon,148,864,"4,241",2703,143,"8,101",36.12%
12,VI,38,115,435,316,5,909,4.05%
13,VII,38,168,815,635,15,"1,671",7.45%
14,VIII,20,169,813,493,27,"1,522",6.79%
15,Visayas,96,452,2063,1444,47,"4,102",18.29%
16,IX,28,257,"1,242",653,46,"2,226",9.93%
17,X,33,158,765,455,60,"1,471",6.56%
18,XI,14,107,755,351,47,"1,274",5.68%
19,XII,16,90,615,391,65,"1,177",5.25%


_________
## DATA PREPROCESSING STEP 4: MAPPING & CLEANING USING REGION

________
### 4.1: Create the Region Dictionary

In [207]:
def get_region_to_island():
    return {
        # Luzon
        "NCR": "Luzon", "CAR": "Luzon", "I": "Luzon", "II": "Luzon",
        "III": "Luzon", "IV-A": "Luzon", "MIMAROPA": "Luzon", "V": "Luzon",
        # Visayas
        "VI": "Visayas", "VII": "Visayas", "VIII": "Visayas",
        # Mindanao
        "IX": "Mindanao", "X": "Mindanao", "XI": "Mindanao", "XII": "Mindanao",
        "Caraga": "Mindanao", "BARMM": "Mindanao",
    }

________
### 4.2: Include Region Column and drop Totals (Region, Grand Total, Percentages)

In [208]:
def add_island_and_filter(df: pd.DataFrame) -> pd.DataFrame:
    """
    Maps the first column's region codes to Island (Luzon/Visayas/Mindanao),
    drops rows where Island is NaN, returns a new DataFrame.
    """
    region_to_island = get_region_to_island()
    out = df.copy()
    out["Island"] = out.iloc[:, 0].map(region_to_island)
    out = out[out["Island"].notna()].reset_index(drop=True)
    return out


# Run Function
cleaning_step_3_df = add_island_and_filter(cleaning_step_2_df)
cleaning_step_3_df

,Island / Region,9 to 17,18 to 25,26 to 35,36 to 45,46 and above,Total,Percentage\n(%),Island
0,NCR,2,38,149,100,16,305,1.36%,Luzon
1,CAR,24,113,322,211,3,673,3.00%,Luzon
2,I,37,183,487,290,8,"1,005",4.48%,Luzon
3,II,15,53,304,137,6,515,2.30%,Luzon
4,III,20,100,444,274,17,855,3.81%,Luzon
5,IV-A,16,168,"1,039",625,48,"1,896",8.45%,Luzon
6,MIMAROPA,13,57,359,204,4,637,2.84%,Luzon
7,V,23,152,"1,137",862,41,"2,215",9.88%,Luzon
8,VI,38,115,435,316,5,909,4.05%,Visayas
9,VII,38,168,815,635,15,"1,671",7.45%,Visayas


_________
## DATA PREPROCESSING STEP 5: INCLUDING YEAR & QUARTER

In [209]:
def add_year_quarter(df: pd.DataFrame, pdf_path: str) -> pd.DataFrame:
    """
    Extracts Year and Quarter from the PDF file name and adds them as columns.
    Example filename: 2025_4th_Quarter.pdf -> Year=2025, Quarter=4
    """
    fname = pdf_path.split("/")[-1]
    parts = fname.split("_")
    year = parts[0]
    quarter = parts[1][0]  # first char of '1st'/'2nd'/'3rd'/'4th'
    out = df.copy()
    out["Year"] = year
    out["Quarter"] = quarter
    return out


# Run Function
cleaning_step_4_df = add_year_quarter(cleaning_step_3_df, PDF)
cleaning_step_4_df

,Island / Region,9 to 17,18 to 25,26 to 35,36 to 45,46 and above,Total,Percentage\n(%),Island,Year,Quarter
0,NCR,2,38,149,100,16,305,1.36%,Luzon,2025,4
1,CAR,24,113,322,211,3,673,3.00%,Luzon,2025,4
2,I,37,183,487,290,8,"1,005",4.48%,Luzon,2025,4
3,II,15,53,304,137,6,515,2.30%,Luzon,2025,4
4,III,20,100,444,274,17,855,3.81%,Luzon,2025,4
5,IV-A,16,168,"1,039",625,48,"1,896",8.45%,Luzon,2025,4
6,MIMAROPA,13,57,359,204,4,637,2.84%,Luzon,2025,4
7,V,23,152,"1,137",862,41,"2,215",9.88%,Luzon,2025,4
8,VI,38,115,435,316,5,909,4.05%,Visayas,2025,4
9,VII,38,168,815,635,15,"1,671",7.45%,Visayas,2025,4


______________
# ACTUAL APPLICATION

## Single Header

#### NOTES: Change table_index if you want to extract other tables on the same page

In [240]:
PDF = "pantawid_reports/2025_4th_Quarter.pdf"
page_num = 22

extract_table_from_pdf_tabula(PDF, page_num, table_index=1)

Got stderr: 2025-10-29 12:46:46.995 java[80784:2020241] [JRSAppKitAWT markAppIsDaemon] failed. SetApplicationIsDaemon returned -50



,Unnamed: 0,Unnamed: 1,Household with Single,Unnamed: 2,Total,Percentage
0,NaN,Region,Multiple Disability Disability Disability,NaN,Member,%
1,NCR,NaN,"1,874 1,965",39,"2,004",3.97%
2,CAR,NaN,"1,138 1,150",54,"1,204",2.39%
3,I,NaN,"3,667 3,954",148,"4,102",8.13%
4,II,NaN,"2,175 2,233",80,"2,313",4.58%
5,III,NaN,"3,049 3,232",80,"3,312",6.56%
6,IV-A,NaN,"3,905 4,064",148,"4,212",8.35%
7,MIMAROPA,NaN,"2,997 3,010",174,"3,184",6.31%
8,V,NaN,"5,184 5,359",99,"5,458",10.82%
9,VI,NaN,"4,532 4,665",167,"4,832",9.58%


In [241]:
extract_table_from_pdf_pdfplumber(PDF, page_num, table_index=1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,Region,None,None,,Household with,,,Single,,Multiple Disability,None,None,,Total,,,Percentage,
1,None,None,None,None,Disability,None,None,Disability,None,None,None,None,None,Member,None,None,%,None
2,NCR,None,None,"1,874",None,None,"1,965",None,None,39,None,None,"2,004",None,None,3.97%,None,None
3,CAR,None,None,"1,138",None,None,"1,150",None,None,54,None,None,"1,204",None,None,2.39%,None,None
4,I,None,None,"3,667",None,None,"3,954",None,None,148,None,None,"4,102",None,None,8.13%,None,None
5,II,None,None,"2,175",None,None,"2,233",None,None,80,None,None,"2,313",None,None,4.58%,None,None
6,III,None,None,"3,049",None,None,"3,232",None,None,80,None,None,"3,312",None,None,6.56%,None,None
7,IV-A,None,None,"3,905",None,None,"4,064",None,None,148,None,None,"4,212",None,None,8.35%,None,None
8,MIMAROPA,None,None,"2,997",None,None,"3,010",None,None,174,None,None,"3,184",None,None,6.31%,None,None
9,V,None,None,"5,184",None,None,"5,359",None,None,99,None,None,"5,458",None,None,10.82%,None,None


## Multiple Headers/ Subheaders

#### NOTES: Change table_index if you want to extract other tables on the same page

In [242]:
PDF = "pantawid_reports/2025_4th_Quarter.pdf"
page_num = 22

pdfplumber_df = extract_table_from_pdf_pdfplumber(PDF, page_num, table_index=1)
step1 = combine_every_3_columns(pdfplumber_df)
ncr_idx, before_ncr = get_rows_before_ncr(step1)
counts = count_non_empty_after_first(before_ncr)
headers = build_headers(before_ncr, counts)
step2 = apply_headers(step1, ncr_idx, headers)
step3 = add_island_and_filter(step2)
final_df = add_year_quarter(step3, PDF)
final_df

,Region,Disability,Disability,Multiple Disability,Member,%,Island,Year,Quarter
0,NCR,"1,874","1,965",39,"2,004",3.97%,Luzon,2025,4
1,CAR,"1,138","1,150",54,"1,204",2.39%,Luzon,2025,4
2,I,"3,667","3,954",148,"4,102",8.13%,Luzon,2025,4
3,II,"2,175","2,233",80,"2,313",4.58%,Luzon,2025,4
4,III,"3,049","3,232",80,"3,312",6.56%,Luzon,2025,4
5,IV-A,"3,905","4,064",148,"4,212",8.35%,Luzon,2025,4
6,MIMAROPA,"2,997","3,010",174,"3,184",6.31%,Luzon,2025,4
7,V,"5,184","5,359",99,"5,458",10.82%,Luzon,2025,4
8,VI,"4,532","4,665",167,"4,832",9.58%,Visayas,2025,4
9,VII,"3,104","3,145",174,"3,319",6.58%,Visayas,2025,4
